In [3]:
import sys
sys.path.append("./yolov5")

In [27]:
import torch
import json
import warnings
from yolov5.models.experimental import attempt_load
from yolov5.models.common import Conv
from yolov5.models.yolo import Detect
import torch.nn as nn
import onnx
import onnxsim
import mo.main as model_optimizer
import subprocess
import blobconverter
import numpy as np

In [8]:
weights = "best.pt"
fn = "model"
f_onnx = f"./{fn}.onnx"
f_simplified = f"./{fn}-simplified.onnx" 
dir_ov = "./output/"
imgsz = 416

In [9]:
# based on export.py
model = attempt_load(weights)  # load FP32 model
nc, names = model.nc, model.names  # number of classes, class names

# Checks
opset = 12
assert nc == len(names), f'Model class count {nc} != len(names) {len(names)}'

# Input
gs = int(max(model.stride))  # grid size (max stride)

# Image size check
if isinstance(imgsz, int):
    imgsz = [imgsz, imgsz]
for sz in imgsz:
    if sz % gs != 0:
        raise ValueError(f"Image size is not a multiple of maximum stride {gs}")

if len(imgsz) != 2:
    raise ValueError(f"Image size must be of length 1 or 2.")
        
im = torch.zeros(1, 3, *imgsz)#.to(device)  # image size(1,3,320,192) BCHW iDetection

# Update model
#im, model = im.half(), model.half()  # to FP16
model.eval()
for k, m in model.named_modules():
    if isinstance(m, Conv):  # assign export-friendly activations
        if isinstance(m.act, nn.SiLU):
            m.act = SiLU()
    elif isinstance(m, Detect):
        m.inplace = inplace
        m.onnx_dynamic = False
        if hasattr(m, 'forward_export'):
            m.forward = m.forward_export  # assign custom forward (optional)
            
for _ in range(2):
    y = model(im)  # dry runs


Fusing layers... 
Model Summary: 213 layers, 7020913 parameters, 0 gradients


In [10]:
print(f"Generating ONNX")
torch.onnx.export(model, im, f_onnx, verbose=False, opset_version=12,
                  training=torch.onnx.TrainingMode.EVAL,
                  do_constant_folding=True,
                  input_names=['images'],
                  output_names=['output'],
                  dynamic_axes=None)

print(f"Checking exported ONNX")
# Checks
model_onnx = onnx.load(f_onnx)  # load onnx model
onnx.checker.check_model(model_onnx)  # check onnx model
# LOGGER.info(onnx.helper.printable_graph(model_onnx.graph))  # print

print(f"Simplifying")
onnx_model, check = onnxsim.simplify(model_onnx)
assert check, 'assert check failed'
#onnx.save(model_onnx, f)

Generating ONNX


/home/matija/Luxonis/model-export/yolo/./yolov5/models/yolo.py:57: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.onnx_dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


Checking exported ONNX
Simplifying


In [11]:
conv_indices = []
for i, n in enumerate(onnx_model.graph.node):
    if "Conv" in n.name:
        conv_indices.append(i)

input1, input2, input3 = conv_indices[-3:]

sigmoid1 = onnx.helper.make_node(
    'Sigmoid',
    inputs=[onnx_model.graph.node[input1].output[0]],
    outputs=['output1_yolov5'],
)

sigmoid2 = onnx.helper.make_node(
    'Sigmoid',
    inputs=[onnx_model.graph.node[input2].output[0]],
    outputs=['output2_yolov5'],
)

sigmoid3 = onnx.helper.make_node(
    'Sigmoid',
    inputs=[onnx_model.graph.node[input3].output[0]],
    outputs=['output3_yolov5'],
)

onnx_model.graph.node.append(sigmoid1)
onnx_model.graph.node.append(sigmoid2)
onnx_model.graph.node.append(sigmoid3)

onnx.save(onnx_model, f_simplified)

In [12]:
model_onnx = onnx.load(f_simplified)  # load onnx model
onnx.checker.check_model(model_onnx)  # check onnx model

In [13]:
# TODO replace this da pogledaš v export.py

In [16]:
import openvino.inference_engine as ie

print(f'Starting export with openvino {ie.__version__}...')

cmd = f"mo --input_model {f_simplified} " \
f"--output_dir {dir_ov} " \
f"--model_name {fn} " \
'--data_type FP16 ' \
'--reverse_input_channel ' \
'--scale 255 ' \
'--output "output1_yolov5,output2_yolov5,output3_yolov5"'

cmd

Starting export with openvino 2021.4.2-3976-0943ed67223-refs/pull/539/head...


'mo --input_model ./model-simplified.onnx --output_dir ./output/ --model_name model --data_type FP16 --reverse_input_channel --scale 255 --output "output1_yolov5,output2_yolov5,output3_yolov5"'

In [19]:
subprocess.check_output(cmd, shell=True)

[ WARNING ]  Const node 'Resize_118/Add_input_port_1/value214810275' returns shape values of 'float64' type but it must be integer or float32. During Elementwise type inference will attempt to cast to float32
[ WARNING ]  Const node 'Resize_140/Add_input_port_1/value218210278' returns shape values of 'float64' type but it must be integer or float32. During Elementwise type inference will attempt to cast to float32
[ WARNING ]  Changing Const node 'Resize_118/Add_input_port_1/value214810512' data type from float16 to <class 'numpy.float32'> for Elementwise operation
[ WARNING ]  Changing Const node 'Resize_140/Add_input_port_1/value218210473' data type from float16 to <class 'numpy.float32'> for Elementwise operation


b"Model Optimizer arguments:\nCommon parameters:\n\t- Path to the Input Model: \t/home/matija/Luxonis/model-export/yolo/./model-simplified.onnx\n\t- Path for generated IR: \t/home/matija/Luxonis/model-export/yolo/./output/\n\t- IR output name: \tmodel\n\t- Log level: \tERROR\n\t- Batch: \tNot specified, inherited from the model\n\t- Input layers: \tNot specified, inherited from the model\n\t- Output layers: \toutput1_yolov5,output2_yolov5,output3_yolov5\n\t- Input shapes: \tNot specified, inherited from the model\n\t- Mean values: \tNot specified\n\t- Scale values: \tNot specified\n\t- Scale factor: \t255.0\n\t- Precision of IR: \tFP16\n\t- Enable fusing: \tTrue\n\t- Enable grouped convolutions fusing: \tTrue\n\t- Move mean values to preprocess section: \tNone\n\t- Reverse input channels: \tTrue\nONNX specific parameters:\n\t- Inference Engine found in: \t/home/matija/Luxonis/envs/base/lib/python3.8/site-packages/openvino\nInference Engine version: \t2021.4.2-3976-0943ed67223-refs/pull

In [77]:
binfile = f"./output/{fn}.bin"
xmlfile = f"./output/{fn}.xml"

blob_path = blobconverter.from_openvino(
    xml=xmlfile,
    bin=binfile,
    data_type="FP16",
    shaves=6,
    version="2021.4",
    use_cache=False,
    output_dir="."
)

[===========================================       ]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# generate json

In [63]:
anchors, sides = [], []

m = model.module.model[-1] if hasattr(model, 'module') else model.model[-1]
for i in range(3):
    sides.append(m.anchor_grid[i].size()[2])
    for j in range(3):
        anchors.extend(m.anchor_grid[i][0, j, 0, 0].numpy())
        #print(np.round(m.anchor_grid[i][0, j, 0, 0].numpy()))

In [64]:
anchors

[76.5,
 6.4257812,
 23.3125,
 25.875,
 65.8125,
 20.921875,
 155.75,
 10.5546875,
 31.6875,
 52.625,
 295.5,
 8.71875,
 58.0625,
 50.65625,
 343.0,
 19.953125,
 168.625,
 60.15625]

In [68]:
sides.sort()
sides[::-1]

[52, 26, 13]

In [54]:
f = open("json/yolov5.json")
content = json.load(f)

In [71]:
masks = dict()
for i, num in enumerate(sides[::-1]):
    masks[f"side{num}"] = list(range(i*3, i*3+3))
masks

{'side52': [0, 1, 2], 'side26': [3, 4, 5], 'side13': [6, 7, 8]}

In [74]:
content["nn_config"]["input_size"] = "x".join([str(x) for x in imgsz])
content["nn_config"]["NN_specific_metadata"]["classes"] = model.nc
content["nn_config"]["NN_specific_metadata"]["anchors"] = anchors
content["nn_config"]["NN_specific_metadata"]["anchor_masks"] = masks
content["mappings"]["labels"] = model.names

In [76]:
content

{'nn_config': {'output_format': 'detection',
  'NN_family': 'YOLO',
  'input_size': '416x416',
  'NN_specific_metadata': {'classes': 4,
   'coordinates': 4,
   'anchors': [76.5,
    6.4257812,
    23.3125,
    25.875,
    65.8125,
    20.921875,
    155.75,
    10.5546875,
    31.6875,
    52.625,
    295.5,
    8.71875,
    58.0625,
    50.65625,
    343.0,
    19.953125,
    168.625,
    60.15625],
   'anchor_masks': {'side52': [0, 1, 2],
    'side26': [3, 4, 5],
    'side13': [6, 7, 8]},
   'iou_threshold': 0.5,
   'confidence_threshold': 0.5}},
 'mappings': {'labels': ['D00', 'D01', 'D02', 'D03']}}

In [ ]:
# TODO remove files